In [ ]:
!python -m spacy download en_core_web_lg

In [1]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
import json
import PyPDF2
from PyPDF2 import PdfReader
import re

ce_nlp = spacy.blank("en")  # a new spacy model
db = DocBin() # a new DocBin object

In [8]:
annotFile = open(r'C:\Users\abhijat_thakare\Desktop\Semicolons\CriteriaExtractor\Resources\annotations.json')
TRAIN_DATA = json.load(annotFile)

In [14]:
for text, annot in tqdm(TRAIN_DATA['annotations']): 
    doc = ce_nlp.make_doc(text) 
    ents = []
    for start, end, label in annot["entities"]:
        span = doc.char_span(start, end, label=label)
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents 
    db.add(doc)

db.to_disk("./training_data.spacy") # save the docbin object

100%|██████████| 5/5 [00:00<00:00,  6.70it/s]


In [12]:
! python -m spacy init config config.cfg --lang en --pipeline ner --optimize efficiency

[!] To generate a more effective transformer-based config (GPU-only),
install the spacy-transformers package and re-run this command. The config
generated now does not use transformers.
[i] Generated config template specific for your use case
- Language: en
- Pipeline: ner
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
[+] Auto-filled config with all values
[+] Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [15]:
! python -m spacy train config.cfg --output ./ --paths.train ./training_data.spacy --paths.dev ./training_data.spacy

[i] Saving to output directory: .
[i] Using CPU

=========================== Initializing pipeline ===========================
[+] Initialized pipeline

============================= Training pipeline =============================
[i] Pipeline: ['tok2vec', 'ner']
[i] Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00   1880.60    0.00    0.00    0.00    0.00
 20     200      24429.61  87397.64  100.00  100.00  100.00    1.00
 40     400          0.00      0.00  100.00  100.00  100.00    1.00
 60     600          0.00      0.00  100.00  100.00  100.00    1.00
 80     800          0.00      0.00  100.00  100.00  100.00    1.00
100    1000          0.00      0.00  100.00  100.00  100.00    1.00
120    1200          0.00      0.00  100.00  100.00  100.00    1.00
140    1400          0.00      0.00  100.00  100.00  100.00    1.00
160    1600          0.00 

[2023-02-23 23:32:33,984] [INFO] Set up nlp object from config
[2023-02-23 23:32:33,993] [INFO] Pipeline: ['tok2vec', 'ner']
[2023-02-23 23:32:33,996] [INFO] Created vocabulary
[2023-02-23 23:32:33,997] [INFO] Finished initializing nlp object
[2023-02-23 23:32:51,605] [INFO] Initialized pipeline components: ['tok2vec', 'ner']


In [2]:
ce_nlp_ner = spacy.load("./model-best") 

In [3]:
def all_work(nlp_text):
    doc = ce_nlp_ner(nlp_text)
    spacy.displacy.render(doc, style="ent", jupyter=True) # display in Jupyter
    lis = []
    for entity in doc.ents:
        lis.append({entity.label_ : entity})
    return lis

In [4]:
def GenerateTextOutput(pdf_path):
    
    ExtractedPDFText = ""
    reader = PdfReader(pdf_path)
    number_of_pages = len(reader.pages)
    for i in range(number_of_pages) :
        page = reader.pages[i]
        ExtractedPDFText += page.extract_text()
        
    New_text = re.sub(r'https?://\S+|www\.\S+', '', ExtractedPDFText) #Remove links
    New_text = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', New_text) #Remove links
    New_text = re.sub(r'\S+\.gov\S*\b', '', New_text) #Remove links
    New_text = re.sub(r'Page\s+\d+\s+of\s+\d+', '', New_text) #Remove page numbers


    #Replace Symbols

    New_text = New_text.replace("<", " < ")
    New_text = New_text.replace(">", " > ")
    New_text = New_text.replace("≥", " >= ")
    New_text = New_text.replace("≤", " <= ")
    New_text = New_text.replace("…", " ... ")
    New_text = New_text.replace("•", " ")
    New_text = New_text.replace("—", " ")
    New_text = New_text.replace("’", " ")
    New_text = New_text.replace("°", " ")
    New_text = New_text.replace("∀", " ")
    New_text = New_text.replace("∃", " ")
    New_text = New_text.replace("é", "e")
    New_text = New_text.replace("”", " ")
    New_text = New_text.replace("ä", " ")
    New_text = New_text.replace(":", " : ")
    New_text = New_text.replace(",", " , ")
    New_text = New_text.replace("^", " ")
    New_text = New_text.replace("ï", "i")
    New_text = New_text.replace("–", " - ")
    New_text = New_text.replace("ö", "o")
    New_text = New_text.replace("“", " ")
    
    return New_text
    

In [10]:
res = GenerateTextOutput(r'C:\Users\abhijat_thakare\Desktop\Converted_NCT04716946.pdf')

In [11]:
newLis = all_work(res)

In [12]:
newLis

[{'INCLUSIONCRITERIA': Inclusion Criteria :   
   Capable of giving signed informed consent which includes compliance with the requirements  
  Outcome Measures  
   Go 
    
  to 
    
    
  Eligibility Criteria  
   Go 
    
  to 
    
    
  Treating Early -stage Non -Small Cell Lung Cancer With Durvalumab and Radiation Therapy - Full Text View - ClinicalTr   09/02/23 ,  19 : 27  
     and restrictions listed in the informed consent form (ICF) and in this protocol. Written informed 
  consent and any locally required authorization obtained from the patient/legal representative 
  prior to performing any protocol - related procedures.  
  Patient a ge  >=  18 at time of consent  
  Early stage NSCLC (Stage I to IIIA; T1 -4 excluding patients with satellite nodules in the same 
  or ipsilateral lobes ,  N0; AJCC 8th edition)  
   Ineligible for or unwilling to undergo surgical resection. Reasons for surgical ineligib ility 
  include :  medically inoperable or surgically unresectable